In [1]:
import numpy as np
import tensorflow as tf
import time
# import random

2022-01-13 19:45:56.381663: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
is_cuda_gpu_min_3

In [2]:
train = np.load('../data/freebase15k_numpy/train.npy')
validation = np.load('../data/freebase15k_numpy/validation.npy')
test = np.load('../data/freebase15k_numpy/test.npy')

entities = np.load('../data/freebase15k_numpy/entities.npy')
relations = np.load('../data/freebase15k_numpy/relations.npy')

train = tf.convert_to_tensor(train.astype(dtype=np.int16), dtype=tf.int32)
validation = tf.convert_to_tensor(validation.astype(dtype=np.int16), dtype=tf.int32)
test = tf.convert_to_tensor(test.astype(dtype=np.int16), dtype=tf.int32)
relations = tf.convert_to_tensor(relations.astype(dtype=np.int16), dtype=tf.int16)
entities = tf.convert_to_tensor(entities.astype(dtype=np.float64), dtype=tf.float64)

2022-01-13 19:46:00.383469: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-13 19:46:00.384712: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-13 19:46:00.413294: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-01-13 19:46:00.413332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sogol
2022-01-13 19:46:00.413355: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sogol
2022-01-13 19:46:00.413444: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-01-13 19:46:00.413477: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-01-13 19:46:00.413488: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] ker

In [3]:
iteration = tf.constant(500)
dim = tf.constant(100)
kns = tf.constant(5)
alpha = tf.Variable(0.000001, dtype = tf.float64)
beta = tf.Variable(0.0000001, dtype = tf.float64)
z = tf.constant(0.5, dtype = tf.float64)
nn1 = tf.random.uniform( shape=(entities.shape[0], dim),
    #   minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
    #   maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn1')
      
nn0 =tf.random.uniform(shape=(entities.shape[0], dim), 
    #   minval=tf.math.truediv(z , tf.cast(dim,tf.float64),'minval'), 
    #   maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn0')
nn2 = tf.random.uniform(shape=(relations.shape[0], dim), 
    #   minval=tf.math.truediv(z , tf.cast(dim,tf.float64)),'minval'), 
    #   maxval=tf.math.truediv(z , tf.cast(dim,tf.float64),'maxval'),
      dtype=tf.dtypes.float64, seed=None, name='nn2')

In [4]:
# @tf.function
def calculateRank(triple):  
    start = time.time() 
    selectedEntities = tf.cast(tf.gather(entities, 0, axis=1), dtype=tf.int32)
    _head_index = tf.gather(triple,0)
    _tail_index = tf.gather(triple,1)
    _relation_index = tf.gather(triple,2)  
    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 1, axis=1) == _tail_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 1, axis=1) == _tail_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 1, axis=1) == _tail_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 0, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 

    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))
    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedH = tf.concat([  tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _tail_index) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedH = tf.concat([couraptedH, tf.reshape(triple, (1,3))], 0)
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.gather(nn1,tf.gather(couraptedH,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( couraptedH,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    rankH = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )

    couraptedH = None
    indexes = None
    existEntites = None
    selectedEntitiesTrain = None
    selectedEntitiesTest = None
    selectedEntitiesvalidation = None
    selectedEntitiesFinal = None
    p = None

    indexes = tf.where( tf.math.logical_and( (tf.gather(train, 0, axis=1) == _head_index) , (tf.gather(train, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(train, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTrain = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(test, 0, axis=1) == _head_index) , (tf.gather(test, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(test, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesTest = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    indexes = tf.where( tf.math.logical_and( (tf.gather(validation, 0, axis=1) == _head_index) , (tf.gather(validation, 2, axis=1) == _relation_index)))
    indexes = tf.reshape(indexes, (indexes.shape[0] ))
    existEntites = tf.gather(tf.gather(validation, indexes), 1, axis=1)
    shape = tf.constant([entities.shape[0]])
    scatter = tf.scatter_nd(tf.reshape(existEntites ,  (existEntites.shape[0] , 1) ), tf.math.add(existEntites ,1), shape)
    selectedEntitiesvalidation = tf.where(tf.math.greater(tf.subtract(selectedEntities , scatter), -1)) 
    selectedEntitiesTrain= tf.reshape(selectedEntitiesTrain, (1, selectedEntitiesTrain.shape[0]))
    selectedEntitiesTest = tf.reshape(selectedEntitiesTest, (1, selectedEntitiesTest.shape[0]))
    selectedEntitiesvalidation = tf.reshape(selectedEntitiesvalidation, (1, selectedEntitiesvalidation.shape[0]))

    selectedEntitiesFinal = tf.sets.intersection(selectedEntitiesTrain, selectedEntitiesTest)
    selectedEntitiesFinal = tf.cast(tf.sets.intersection(selectedEntitiesFinal, selectedEntitiesvalidation), dtype=tf.int32).values
    couraptedT = tf.concat([   tf.fill([selectedEntitiesFinal.shape[0], 1], _head_index) , tf.reshape(selectedEntitiesFinal, (selectedEntitiesFinal.shape[0], 1)) , tf.fill([selectedEntitiesFinal.shape[0], 1], _relation_index)],1)
    couraptedT = tf.concat([couraptedT, tf.reshape(triple, (1,3))], 0)
    p =  tf.linalg.diag_part(tf.math.sigmoid(tf.tensordot(tf.gather(nn1,tf.gather(couraptedT,1, axis=1),0),tf.transpose(  tf.gather( nn0,  tf.gather( couraptedT,0, axis=1)) + tf.gather(nn2,_relation_index) ),axes=1 )))
    rankT = tf.where(tf.math.top_k( p ,k = p.shape[0]).indices  == tf.subtract(p.shape[0] , 1) )
    # print(rankH, rankT)
    if tf.math.greater(rankH , rankT):
        return rankT
    else:
        return rankH
    

In [5]:
def mrr(ranks):
    inverse = []
    one = tf.constant(1.0, dtype = tf.float64)
    for rank in ranks:
        inverse.append( tf.math.truediv(one , tf.cast(rank,tf.float64)))
    summ = tf.reduce_sum(inverse)
    return tf.math.multiply(tf.math.truediv(one , len(inverse)) , summ)

In [6]:
start = time.time()
paddings = tf.constant([[0, 1,], [0, 0]])
t_row = train.shape[0]
for x in range(0 , iteration):
    tf.random.shuffle(train, seed=None, name=None)
    with tf.device('/cpu:0'):
        total_samples  = tf.gather( tf.random.categorical([tf.gather(entities, 2, axis=1)], tf.math.multiply(kns,t_row) , dtype=None, seed=None),0)
    for tIndex in range(0 , t_row):
        triple = tf.gather(train, tIndex)
        _head_index = tf.gather(triple,0)
        _relation_index = tf.gather(triple,2)
        _vHead = tf.gather(nn0,_head_index)
        _vRel = tf.gather(nn2,_relation_index)

        samples = [tf.slice(total_samples,begin=[tf.math.multiply(tIndex, kns)],size=[kns])]
        samples = tf.pad(samples, paddings, constant_values=0)
        samples =  tf.cast(samples, tf.int64)         
        samples = tf.transpose(tf.concat([samples, [[tf.gather(triple,1)],[1]]], 1))
        indices = tf.gather(samples, 0, axis=1)
        _nn1_samples = tf.gather(nn1, indices)
        _sigmoid =tf.math.sigmoid(tf.tensordot( _nn1_samples , tf.transpose(tf.math.add(_vHead , _vRel)) , axes=1))
        cost = tf.math.subtract(tf.cast(tf.gather(samples, 1, axis=1), tf.float64) , _sigmoid)
        g = tf.math.multiply(alpha , cost)
        g1 = tf.math.multiply(beta , cost)
        _nn1_samples = tf.math.add(tf.math.multiply(tf.gather(nn1, indices) , tf.reshape(g, (6, 1))) , _nn1_samples)
        nn1 = tf.tensor_scatter_nd_update(nn1,tf.expand_dims(indices, 1),_nn1_samples)

        _nn2_sample = tf.math.add_n([tf.math.reduce_sum(tf.math.multiply(_vRel , tf.reshape(g1, (g1.shape[0], 1))), axis=0, keepdims=False)  , _vRel, tf.math.reduce_sum(_nn1_samples, axis=0, keepdims=False)])
        indices = tf.constant(_relation_index)
        nn2 = tf.tensor_scatter_nd_update(nn2,tf.expand_dims([indices], 1), tf.reshape(_nn2_sample,(1, _nn2_sample.shape[0])))
  
        indices = tf.constant(_head_index)
        _nn0_sample = tf.math.add(tf.math.reduce_sum(tf.math.multiply(_vHead , tf.reshape(g, (g.shape[0], 1))), axis=0, keepdims=False)  , _nn2_sample)
        nn0 = tf.tensor_scatter_nd_update(nn0,tf.expand_dims([indices], 1), tf.reshape(_nn0_sample,(1, _nn0_sample.shape[0])))
    print(time.time() - start)        
    if x % 5 == 0 and x > 0:
        tensor = []
        for tIndex in range(0 , 100):
            triple = tf.gather(validation,tIndex)
            tensor.append(calculateRank(triple))
        print(mrr(tensor))
        print(time.time() - start)
# _mrrList = []
# # for triple in validation:
# _mrrList.append(calculateRank(validation[0,:]))
# print('total one sample calc time =>  ' , time.time() - start) 

4154.391008853912
7896.03627204895
11637.442919254303
15391.977689266205
19151.181975364685
23077.842121362686


2022-01-14 02:10:54.272803: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1669264200 exceeds 10% of free system memory.
2022-01-14 02:10:55.320808: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1669264200 exceeds 10% of free system memory.
2022-01-14 02:10:55.955703: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1788020000 exceeds 10% of free system memory.
2022-01-14 02:10:57.086025: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1788020000 exceeds 10% of free system memory.
2022-01-14 02:10:57.740621: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1733898272 exceeds 10% of free system memory.


tf.Tensor(6.955969549680422e-05, shape=(), dtype=float64)
23457.55275464058
27631.08286380768
31799.847880601883
35884.12597465515
40615.26541471481
45255.61574625969
tf.Tensor(6.955969549680422e-05, shape=(), dtype=float64)
45621.06079506874
50328.17315411568
55340.94731211662
59849.16982507706
64352.936138391495
68831.87132549286
tf.Tensor(6.955969549680422e-05, shape=(), dtype=float64)
69205.92372083664
73695.4770348072
78193.16778039932
82366.62028646469
86376.95954227448


KeyboardInterrupt: 

In [ ]:
tensor = []
for tIndex in range(0 , 15):
    triple = tf.gather(validation,tIndex)
    tensor.append(calculateRank(triple))
print(mrr(tensor))
# tensor = tf.zeros(validation.shape[0], tf.int64)

# c = lambda i: tf.less(i, validation.shape[0])
# i = tf.constant(0)
# b = lambda i: (calculateRank,tf.gather(validation,i),)
# # print(tf.gather(validation,i))
# r = tf.while_loop(c, b, loop_vars= [i])
# print(tensor)